Runs two basic convolutional networks on the pilot dataset.

### Things to do next
- Figure out how to do regression (see below).
- ~~Figure out how to do multitask learning (i.e. try to predict the different reps and different genes).~~
- ~~Run on the scale up dataset.~~
- Read up on the "Interpreting a DragoNN model using filter visualization" and "Interpreting data with a DragoNN model" in the Dragonn tutorial.

### Installing Dragonn
- Clone from https://github.com/kundajelab/dragonn
- ```python setup.py```
    - I needed to ```brew install geos```

In [1]:
from dragonn import models

from collections import OrderedDict
from pprint import pprint
from warnings import warn

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Using Theano backend.
/Users/andrewlamb/anaconda/lib/python2.7/site-packages/Theano-0.8.2-py2.7.egg/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


Read in the data.
- Samples are sequences that will be one hot encoded
- Try and predict the normalized values. **I couldn't figure out how to do regression with Dragonn, so I just rounded the values to 0 or 1 based on the median.**

In [2]:
key_to_seq = OrderedDict()

with open("../data/Scaleup_counts_sequences/ScaleUpDesign1.sequences.txt") as f:
    for line in f:
        key, seq = line.strip().split()
        
        # TODO: Figure out if this is an OK thing to do. 'N' basically means the 
        # sequencing software couldn't figure out what the base was...?
        if "N" in seq:
            warn("Replacing 'N' bases in seq with 'A' in seq {}.".format(seq))
            seq = seq.replace("N", "A")
        
        assert key not in key_to_seq
        key_to_seq[key] = seq
        
with open("../data/Scaleup_counts_sequences/ScaleUpDesign2.sequences.txt") as f:
    for line in f:
        key, seq = line.strip().split()
        
        if "N" in seq:
            warn("Replacing 'N' bases in seq with 'A' in seq {}.".format(seq))
            seq = seq.replace("N", "A")
        
        assert key not in key_to_seq
        key_to_seq[key] = seq
        
pprint(key_to_seq.items()[:5])

print "{} total sequences of length {}".format(len(key_to_seq), len(key_to_seq.values()[0]))

/Users/andrewlamb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNNNNNNGATCCCGCGGCCGTGTTTTCCTGGTGGCCCGGCCGTGCCTGAGGTTTCTCCCCGAGCCGCCGCCTCTGCGGGCTCCCGGGTGCCCTTGCCCTCACGGTCCCCGGCCCTCGCCCGTCTGTGCCCTCTTCCCCGCCCG.
/Users/andrewlamb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNGATCCCGCGGCCGTGTTTTCCTGGTGGCCCGGCCGTGCCTGAGGTTTCTCCCCGAGCCGCCGCCTCTGCGGGCTCCCGGGTGCCCTTGCCCTCACGGTCCCCGGCCCTCGCCCGTCTGTGCCCTCTTCCCCGCCCGCCGCC.
/Users/andrewlamb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNNNNNNNNNNNNNNNGAATTCAATGGAGTGGACTGGAGTGCTGTGGGGTGGAGTGGAATGGAGTGTAGTTGAATGGAGTGGAATGGAATGCGATGGAATGGAGTGGAGTTGAGCAGAGTGAAGTGGAAAGGTGTAGAATGGA.


[('H1hesc_1_0_0_chr20_30310735',
  'GGGAGCCCAGAAGGCGACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCAC'),
 ('H1hesc_1_0_1_chr20_30310735',
  'CCCAGAAGGCGACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCC'),
 ('H1hesc_1_0_2_chr20_30310735',
  'AAGGCGACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCCTTGGC'),
 ('H1hesc_1_0_3_chr20_30310735',
  'GACACAGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCCTTGGCTCTCC'),
 ('H1hesc_1_0_4_chr20_30310735',
  'AGGAATTGCGAAGCTCAGGAACCAGCCCCCTCGCTTGCTTCCTCCTCCATCGCCCGGATCGAGGGCGGCCGCTCCGCAGCCGCGGCCTCCTGCCACCCGGGAGCCCAGCCCCCTCTCTCTTGCACGCCCCTTGGCTCTCCGCCTC')]
487137 total sequences of length 145


/Users/andrewlamb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNNNNNNNNNNGAATTCAATGGAGTGGACTGGAGTGCTGTGGGGTGGAGTGGAATGGAGTGTAGTTGAATGGAGTGGAATGGAATGCGATGGAATGGAGTGGAGTTGAGCAGAGTGAAGTGGAAAGGTGTAGAATGGAATGGA.
/Users/andrewlamb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNNNNNNGAATTCAATGGAGTGGACTGGAGTGCTGTGGGGTGGAGTGGAATGGAGTGTAGTTGAATGGAGTGGAATGGAATGCGATGGAATGGAGTGGAGTTGAGCAGAGTGAAGTGGAAAGGTGTAGAATGGAATGGAATGGA.
/Users/andrewlamb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Replacing 'N' bases in seq with 'A' in seq NNNGAATTCAATGGAGTGGACTGGAGTGCTGTGGGGTGGAGTGGAATGGAGTGTAGTTGAATGGAGTGGAATGGAATGCGATGGAATGGAGTGGAGTTGAGCAGAGTGAAGTGGAAAGGTGTAGAATGGAATGGAATGGAGTGGA.


In [3]:
data = {}
cell_types =  ["HepG2", "K562"]
promoters = ["SV40P", "minP"]
design_names = ["ScaleUpDesign1", "ScaleUpDesign2"]

for cell_type in cell_types:
    for promoter in promoters:
        experiment_key = (cell_type, promoter)

        for design_name in design_names:
            data[experiment_key] = {}

            with open("../data/Scaleup_normalized/{}_{}_{}_mRNA_Rep1.normalized".format(cell_type, design_name, promoter)) as f:
                for line in f:
                    parts = line.strip().split()

                    key = parts[0]
                    val = float(parts[1])
                    if parts[2] == "1":
                        data[experiment_key][key] = val

            with open("../data/Scaleup_normalized/{}_{}_{}_mRNA_Rep2.normalized".format(cell_type, design_name, promoter)) as f:
                for line in f:
                    parts = line.strip().split()

                    key = parts[0]
                    val = float(parts[1])
                    if parts[2] == "1" and key in data[experiment_key]:
                        data[experiment_key][key] = val
            
print "Data from experiment {}:".format(data.items()[0][0])
pprint(data.items()[0][1].items()[:5])

Data from experiment ('HepG2', 'minP'):
[('H1hesc_9_213_18_chr2_26895415', -4.259193703616347),
 ('K562_14_89_10_chr19_14848415', -2.293409418954255),
 ('H1hesc_8_354_3_chr5_174622815', -0.6748385255902782),
 ('H1hesc_5_1094_24_chr4_40311455', -0.1252866601459317),
 ('Huvec_12_379_1_chr14_67894115', -0.5824728799359882)]


In [4]:
# One hot encode DNA sequences the standard way.
bases = ['A', 'T', 'C', 'G']

def one_hot_encode_seq(seq):
    result = np.zeros((len(bases), len(seq)))
    
    for i, base in enumerate(seq):
        result[bases.index(base), i] = 1

    return result

def seqs_to_encoded_matrix(seqs):
    # Wrangle the data into a shape that Dragonn wants.
    result = np.concatenate(
        map(one_hot_encode_seq, seqs)
    ).reshape(
        len(seqs), 1, len(bases), len(seqs[0])
    )
    
    # Check we actually did the encoding right.
    for i in range(len(seqs)):
        for j in range(len(seqs[0])):
            assert sum(result[i, 0, :, j]) == 1
    
    return result

- We formulate this as a multi-task learning problem, where each cell type and promoter combo is a task, i.e. the tasks are 
- Some of the normalized scores are too noisy (as determined by the SHARPR software). For now, we only consider a sequence if it has a good measurement for all of the tasks
  

In [5]:
valid_keys = reduce(
    lambda acc, d: acc.intersection(d.keys()), 
    data.values()[1:], 
    set(data.values()[0].keys())
)

print "{} sequences have measurements for all tasks.".format(len(valid_keys))

129623 sequences have measurements for all tasks.


In [6]:
X = seqs_to_encoded_matrix([key_to_seq[key] for key in valid_keys])

In [7]:
X.shape

(129623, 1, 4, 145)

In [8]:
# Just round to the median, to make this a classification task for now.
experiment_labels = []
for experiment_key, key_to_normalized in data.items():
    
    filtered_normalized = [key_to_normalized[key] for key in valid_keys]
    
    median = np.median(filtered_normalized)
    experiment_labels.append(
        np.array(map(lambda val: val > median, filtered_normalized)).reshape(-1, 1)
    )

y = np.hstack(experiment_labels)

In [9]:
y.shape

(129623, 4)

In [10]:
X[:5, :, :, :]

array([[[[ 1.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  1.,  0., ...,  0.,  0.,  1.],
         [ 0.,  0.,  1., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.]]],


       [[[ 0.,  1.,  1., ...,  1.,  1.,  1.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 1.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  1.,  0.],
         [ 1.,  1.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  1., ...,  1.,  0.,  1.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 1.,  0.,  1., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  1.,  0., ...,  0.,  1.,  1.]]],


       [[[ 1.,  0.,  1., ...,  0.,  1.,  1.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  1.,  0., ...,  0.,  0.,  0.]]]])

In [11]:
y[:5, :]

array([[False,  True,  True, False],
       [ True, False, False,  True],
       [ True, False, False, False],
       [ True,  True, False,  True],
       [ True, False,  True, False]], dtype=bool)

In [12]:
n_valid = 1000
X_train = X[:-n_valid,:,:,:]
y_train = y[:-n_valid,:]

X_valid = X[-n_valid:,:,:,:]
y_valid = y[-n_valid:,:]

Start with the model used in the Dragonn tutorial. Train and plot the train and validation loss.

In [ ]:
model = models.SequenceDNN(
    seq_length=X_train.shape[3],
    num_filters=[1],
    conv_width=[45],
    pool_width=45,
    num_tasks=y_train.shape[1]
)

In [ ]:
model.train(X_train, y_train, (X_valid, y_valid))

In [ ]:
def print_loss(model):
    train_losses, valid_losses = [np.array([epoch_metrics['Loss'] for epoch_metrics in metrics])
                                  for metrics in (model.train_metrics, model.valid_metrics)]

    # Pretty sure early stopping works by taking the mean of losses, might want to double check
    train_losses = train_losses.mean(axis=1)
    valid_losses = valid_losses.mean(axis=1)

    f = plt.figure(figsize=(10, 4))
    ax = f.add_subplot(1, 1, 1)
    
    ax.plot(range(len(train_losses)), train_losses, label='Training',lw=4)
    ax.plot(range(len(train_losses)), valid_losses, label='Validation', lw=4)
    
    min_loss_indx = min(enumerate(valid_losses), key=lambda x: x[1])[0]
    ax.plot([min_loss_indx, min_loss_indx], [0, 1.0], 'k--', label='Early Stop')
    ax.legend(loc="upper right")
    ax.set_ylabel("Loss")
    ax.set_ylim((0.0,1.0))
    ax.set_xlabel("Epoch")
    plt.show()

In [ ]:
print_loss(model)

Train and test on a multi-filter model (not a lot of filters).

In [ ]:
multi_filter_model = models.SequenceDNN(
    seq_length=X_train.shape[3],
    num_filters=[2],
    conv_width=[45],
    pool_width=45,
    dropout=0.1,
    num_tasks=y_train.shape[1]
)

In [ ]:
multi_filter_model.train(X_train, y_train, (X_valid, y_valid))

In [ ]:
print_loss(multi_filter_model)